In [1]:
# -----------------------------
# Step 1: Install and Upgrade Necessary Packages
# -----------------------------
!pip install --upgrade torchcrf seqeval datasets

# -----------------------------
# Step 2: Import Libraries and Set Device
# -----------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from TorchCRF import CRF  # Corrected import statement
from datasets import load_dataset
from sklearn.metrics import classification_report
from typing import List
import numpy as np
from seqeval.metrics import classification_report as seq_classification_report
from torch.utils.data import TensorDataset, DataLoader
from collections import defaultdict
import pickle
from google.colab import files
import nltk
nltk.download('punkt_tab')

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# -----------------------------
# Step 3: Define the Custom LSTM and LSTM-CRF Model
# -----------------------------
class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CustomLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Input gate parameters
        self.W_ii = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.W_hi = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_i = nn.Parameter(torch.Tensor(hidden_size))

        # Forget gate parameters
        self.W_if = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.W_hf = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(hidden_size))

        # Cell gate parameters
        self.W_ig = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.W_hg = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_g = nn.Parameter(torch.Tensor(hidden_size))

        # Output gate parameters
        self.W_io = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.W_ho = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_o = nn.Parameter(torch.Tensor(hidden_size))

        self.init_weights()

    def init_weights(self):
        # Initialize weight matrices with Xavier Uniform
        nn.init.xavier_uniform_(self.W_ii)
        nn.init.xavier_uniform_(self.W_hi)
        nn.init.xavier_uniform_(self.W_if)
        nn.init.xavier_uniform_(self.W_hf)
        nn.init.xavier_uniform_(self.W_ig)
        nn.init.xavier_uniform_(self.W_hg)
        nn.init.xavier_uniform_(self.W_io)
        nn.init.xavier_uniform_(self.W_ho)

        # Initialize biases with zeros
        nn.init.zeros_(self.b_i)
        nn.init.zeros_(self.b_f)
        nn.init.zeros_(self.b_g)
        nn.init.zeros_(self.b_o)

    def forward(self, input_seq, h_0=None, c_0=None):
        """
        input_seq: Tensor of shape (batch_size, seq_length, input_size)
        h_0: Initial hidden state (batch_size, hidden_size)
        c_0: Initial cell state (batch_size, hidden_size)
        Returns:
            h_seq: Tensor containing hidden states for all time steps (batch_size, seq_length, hidden_size)
            (h_n, c_n): Final hidden and cell states
        """
        batch_size, seq_length, _ = input_seq.size()
        if h_0 is None:
            h_t = torch.zeros(batch_size, self.hidden_size, device=input_seq.device)
        else:
            h_t = h_0
        if c_0 is None:
            c_t = torch.zeros(batch_size, self.hidden_size, device=input_seq.device)
        else:
            c_t = c_0

        h_seq = []

        for t in range(seq_length):
            x_t = input_seq[:, t, :]  # (batch_size, input_size)

            i_t = torch.sigmoid(x_t @ self.W_ii.T + h_t @ self.W_hi.T + self.b_i)
            f_t = torch.sigmoid(x_t @ self.W_if.T + h_t @ self.W_hf.T + self.b_f)
            g_t = torch.tanh(x_t @ self.W_ig.T + h_t @ self.W_hg.T + self.b_g)
            o_t = torch.sigmoid(x_t @ self.W_io.T + h_t @ self.W_ho.T + self.b_o)

            c_t = f_t * c_t + i_t * g_t
            h_t = o_t * torch.tanh(c_t)

            h_seq.append(h_t.unsqueeze(1))  # (batch_size, 1, hidden_size)

        h_seq = torch.cat(h_seq, dim=1)  # (batch_size, seq_length, hidden_size)
        return h_seq, (h_t, c_t)

class BidirectionalCustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BidirectionalCustomLSTM, self).__init__()
        self.forward_lstm = CustomLSTM(input_size, hidden_size)
        self.backward_lstm = CustomLSTM(input_size, hidden_size)
        self.hidden_size = hidden_size

    def forward(self, input_seq, h_0=None, c_0=None):
        # Forward direction
        forward_out, (h_f, c_f) = self.forward_lstm(input_seq, h_0, c_0)

        # Backward direction
        reversed_input = torch.flip(input_seq, [1])  # Reverse the sequence
        backward_out, (h_b, c_b) = self.backward_lstm(reversed_input, h_0, c_0)
        backward_out = torch.flip(backward_out, [1])  # Re-reverse to original order

        # Concatenate forward and backward outputs
        h_seq = torch.cat([forward_out, backward_out], dim=2)  # (batch_size, seq_length, 2*hidden_size)

        # Final hidden and cell states
        h_n = torch.cat([h_f, h_b], dim=1)  # (batch_size, 2*hidden_size)
        c_n = torch.cat([c_f, c_b], dim=1)  # (batch_size, 2*hidden_size)

        return h_seq, (h_n, c_n)

class LSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim, padding_idx, dropout=0.1):
        super(LSTM_CRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.lstm = BidirectionalCustomLSTM(embedding_dim, hidden_dim // 2)  # hidden_dim // 2 for each direction
        self.dropout = nn.Dropout(dropout)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.crf = CRF(tagset_size)

    def forward(self, sentences, tags, mask):
        embeds = self.embedding(sentences)  # (batch_size, seq_length, embedding_dim)
        embeds = self.dropout(embeds)
        lstm_out, _ = self.lstm(embeds)  # (batch_size, seq_length, hidden_dim)

        lstm_out = lstm_out.transpose(0, 1) if lstm_out.shape[0] == sentences.shape[1] else lstm_out

        lstm_out = self.dropout(lstm_out)
        emissions = self.hidden2tag(lstm_out)  # (batch_size, seq_length, tagset_size)

        # Transpose to (seq_length, batch_size, tagset_size)
        emissions = emissions.transpose(0, 1)
        tags = tags.transpose(0, 1)
        mask = mask.transpose(0, 1)

        loss = -self.crf(emissions, tags, mask=mask)
        loss = torch.mean(loss)
        return loss


    def predict(self, sentences, mask):
        self.eval()
        predictions = []
        with torch.no_grad():
          embeds = self.embedding(sentences)
          embeds = self.dropout(embeds)
          lstm_out, _ = self.lstm(embeds)

          lstm_out = lstm_out.transpose(0, 1) if lstm_out.shape[0] == sentences.shape[1] else lstm_out

          lstm_out = self.dropout(lstm_out)
          emissions = self.hidden2tag(lstm_out)

          # Transpose to (seq_length, batch_size, tagset_size)

          # Alp - commenting this out to debug step 7
          emissions = emissions.transpose(0, 1)
          mask = mask.transpose(0, 1)

          predictions = self.crf.viterbi_decode(emissions, mask=mask)  # List[List[int]]
        return predictions  # List[List[int]]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=605b1cae0c459ecffef941193029e4fc179dad05ffebe97ae4b0a7c13bf7f924
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency res

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Using device: cpu


In [2]:
# -----------------------------
# Step 4: Load and Preprocess the CoNLL-2003 Dataset
# -----------------------------
# Load the CoNLL-2003 dataset
dataset = load_dataset('conll2003')

# Inspect the dataset
print(dataset)

# Example of the dataset
print(dataset['train'][0])

# Extract all unique words and tags
words = set()
tags = set()

for split in ['train', 'validation', 'test']:
    for sentence in dataset[split]:
        for word in sentence['tokens']:
            words.add(word.lower())  # Lowercasing for normalization
        for tag in sentence['ner_tags']:
            tags.add(tag)

# Create word2idx dictionary
word2idx = {"<PAD>": 0, "<UNK>": 1}
for word in sorted(words):
    word2idx[word] = len(word2idx)

# Mapping from tag indices to tag names
tag_names = dataset['train'].features['ner_tags'].feature.names
tag2idx = {tag: idx for idx, tag in enumerate(tag_names)}
idx2tag = {v: k for k, v in tag2idx.items()}

print(f"Number of unique words: {len(word2idx)}")
print(f"Number of unique tags: {len(tag2idx)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})
{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}
Number of unique words: 26871
Number of unique tags: 9


In [3]:
# -----------------------------
# Step 5: Encode the Dataset
# -----------------------------
# Parameters
MAX_LEN = 50  # Maximum sentence length
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
BATCH_SIZE = 16
EPOCHS = 5
O_TAG_IDX = tag2idx['O']

# Encoding functions
def encode_sentences(sentences: List[List[str]], word2idx: dict, max_len: int) -> torch.Tensor:
    encoded = []
    for sentence in sentences:
        encoded_sentence = [word2idx.get(word.lower(), word2idx["<UNK>"]) for word in sentence]
        # Padding
        if len(encoded_sentence) < max_len:
            encoded_sentence += [word2idx["<PAD>"]] * (max_len - len(encoded_sentence))
        else:
            encoded_sentence = encoded_sentence[:max_len]
        encoded.append(encoded_sentence)
    return torch.tensor(encoded, dtype=torch.long)

def encode_labels(labels: List[List[int]], max_len: int) -> torch.Tensor:
    encoded = []
    for label_seq in labels:
        # Keep the original label indices
        encoded_label = label_seq.copy()
        if len(encoded_label) < max_len:
            # Pad with 'O' tag index
            encoded_label += [O_TAG_IDX] * (max_len - len(encoded_label))
        else:
            encoded_label = encoded_label[:max_len]
        encoded.append(encoded_label)
    return torch.tensor(encoded, dtype=torch.long)


# Prepare training data
train_sentences = [example['tokens'] for example in dataset['train']]
train_labels = [example['ner_tags'] for example in dataset['train']]

X_train = encode_sentences(train_sentences, word2idx, MAX_LEN)
y_train = encode_labels(train_labels, MAX_LEN)

# Prepare validation data
val_sentences = [example['tokens'] for example in dataset['validation']]
val_labels = [example['ner_tags'] for example in dataset['validation']]

X_val = encode_sentences(val_sentences, word2idx, MAX_LEN)
y_val = encode_labels(val_labels, MAX_LEN)

# Prepare test data (optional, for later evaluation)
test_sentences = [example['tokens'] for example in dataset['test']]
test_labels = [example['ner_tags'] for example in dataset['test']]

X_test = encode_sentences(test_sentences, word2idx, MAX_LEN)
y_test = encode_labels(test_labels, MAX_LEN)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

Training samples: 14041
Validation samples: 3250
Test samples: 3453


In [4]:
# -----------------------------
# Step 6: Create DataLoaders
# -----------------------------
# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
# -----------------------------
# Step 7: Train the LSTM-CRF Model
# -----------------------------
# Initialize Model
VOCAB_SIZE = len(word2idx)
TAGSET_SIZE = len(tag2idx)
PAD_IDX = word2idx["<PAD>"]

model = LSTM_CRF(VOCAB_SIZE, TAGSET_SIZE, EMBEDDING_DIM, HIDDEN_DIM, padding_idx=PAD_IDX)
model.to(device)

# Initialize Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.1)

# Training Loop
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        # Create mask (batch_size, max_len)
        mask = (batch_X != word2idx["<PAD>"]).to(device)


        # Zero gradients
        optimizer.zero_grad()

        # Compute loss
        loss = model(batch_X, batch_y, mask)
        epoch_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}")

Epoch 1/5, Loss: -1829.9644
Epoch 2/5, Loss: -6174.2734
Epoch 3/5, Loss: -10795.0779
Epoch 4/5, Loss: -15645.2595
Epoch 5/5, Loss: -20298.5589


In [6]:
def evaluate_and_show_results_with_metrics(model, X_test, y_test, test_sentences, idx2tag, device, PAD_IDX=0):
    model.eval()
    all_preds = []
    all_true = []

    # Move data to the specified device
    X_test, y_test = X_test.to(device), y_test.to(device)
    mask = (X_test != PAD_IDX)

    with torch.no_grad():
        # Get model predictions
        predictions = model.predict(X_test, mask=mask)  # List[List[int]]

    for i, (pred_indices, true_labels) in enumerate(zip(predictions, y_test)):
        tokens = test_sentences[i]
        true_labels = true_labels.cpu().numpy()

        # Remove padding from true labels
        valid_positions = (true_labels != PAD_IDX)
        true_labels_filtered = true_labels[valid_positions]
        pred_indices = np.array(pred_indices)

        # Remove padding from predicted labels
        # Ensure pred_indices is not longer than true_labels_filtered
        if len(pred_indices) > len(true_labels_filtered):
            pred_indices = pred_indices[:len(true_labels_filtered)]

        # Map indices to tags for true labels and predicted labels
        true_labels_list = [idx2tag[label] for label in true_labels_filtered]
        pred_tags_list = [idx2tag[idx] for idx in pred_indices]

        # Collect true and predicted labels for metrics
        all_true.append(true_labels_list)
        all_preds.append(pred_tags_list)

        # Print tokens with true labels and predicted labels
        print(f"\nSequence {i}:")
        print(f"{'Token':15} {'True Label':15} {'Predicted Label'}")
        print('-' * 45)
        for token, true_label, pred_label in zip(tokens, true_labels_list, pred_tags_list):
            print(f"{token:15} {true_label:15} {pred_label}")

    # Print classification report
    print("\nClassification Report:")
    print(seq_classification_report(all_true, all_preds, zero_division=0))



evaluate_and_show_results_with_metrics(model, X_test, y_test, test_sentences, idx2tag, device)


Sequence 0:
Token           True Label      Predicted Label
---------------------------------------------
SOCCER          B-LOC           O
-               B-PER           B-LOC

Sequence 1:
Token           True Label      Predicted Label
---------------------------------------------
Nadim           B-PER           O
Ladki           I-PER           B-MISC

Sequence 2:
Token           True Label      Predicted Label
---------------------------------------------
AL-AIN          B-LOC           O
,               B-LOC           I-LOC
United          I-LOC           B-PER
Arab            I-LOC           O

Sequence 3:
Token           True Label      Predicted Label
---------------------------------------------
Japan           B-LOC           O
began           B-MISC          I-LOC
the             I-MISC          B-MISC
defence         B-LOC           O

Sequence 4:
Token           True Label      Predicted Label
---------------------------------------------
But             B-LOC          

In [7]:
# -----------------------------
# Step 8: Save the Model and Dictionaries
# -----------------------------
# Save the model state dictionary
torch.save(model.state_dict(), 'custom_lstm_crf_model.pth')
print("Model saved successfully!")

# Save word2idx and tag2idx dictionaries using pickle
with open('word2idx.pkl', 'wb') as f:
    pickle.dump(word2idx, f)
print("word2idx saved successfully!")

with open('tag2idx.pkl', 'wb') as f:
    pickle.dump(tag2idx, f)
print("tag2idx saved successfully!")

# Optional: Download the files to your local machine
# Uncomment the lines below if you wish to download the files immediately
# files.download('custom_lstm_crf_model.pth')
# files.download('word2idx.pkl')
# files.download('tag2idx.pkl')

Model saved successfully!
word2idx saved successfully!
tag2idx saved successfully!


In [15]:
import nltk
nltk.download('punkt_tab')
# -----------------------------
# Step 9: Upload Your Text File to Colab
# -----------------------------
from google.colab import files
import nltk
from nltk.tokenize import sent_tokenize

# Upload the file
uploaded = files.upload()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Saving combined_summaries.txt to combined_summaries.txt


In [11]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [30]:
# -----------------------------
# Step 10: Load and Preprocess Your Text File
# -----------------------------

# Ensure that the 'punkt' resource is downloaded

file_path = "combined_summaries.txt"

def load_text_file(file_path: str) -> list:
    """
    Load text from a file and split it into sentences.

    Args:
        file_path (str): Path to the text file.

    Returns:
        list: A list of sentences.
    """
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    sentences = sent_tokenize(text)
    return sentences

# Get the uploaded file name
file_path = list(uploaded.keys())[0]  # Get the first uploaded file name

# Load and preprocess the text file
sentences = load_text_file(file_path)
print(f"Number of sentences loaded: {len(sentences)}")
print(f"First 3 sentences: {sentences[:3]}")


def tokenize_sentences(sentences: List[str]) -> List[List[str]]:
    """
    Tokenize each sentence into words.

    Args:
        sentences (List[str]): A list of sentences.

    Returns:
        List[List[str]]: A list of tokenized sentences.
    """
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
    return tokenized_sentences

def encode_sentences(sentences: List[List[str]], word2idx: dict, max_len: int) -> torch.Tensor:
    """
    Encode sentences into tensor of word indices with padding/truncation.

    Args:
        sentences (List[List[str]]): Tokenized sentences.
        word2idx (dict): Mapping from words to indices.
        max_len (int): Maximum sentence length.

    Returns:
        torch.Tensor: Tensor of shape (num_sentences, max_len).
    """
    encoded = []
    for sentence in sentences:
        encoded_sentence = [word2idx.get(word.lower(), word2idx.get("<UNK>", 0)) for word in sentence]  # Default to <UNK> if word is missing
        if len(encoded_sentence) < max_len:
            encoded_sentence += [word2idx.get("<PAD>", 0)] * (max_len - len(encoded_sentence))  # Ensure <PAD> exists
        else:
            encoded_sentence = encoded_sentence[:max_len]
        encoded.append(encoded_sentence)
    return torch.tensor(encoded, dtype=torch.long)

def encode_labels(labels: List[List[str]], tag2idx: dict, max_len: int) -> torch.Tensor:
    """
    Encode labels into tensor of tag indices with padding/truncation.

    Args:
        labels (List[List[str]]): Tokenized labels corresponding to sentences.
        tag2idx (dict): Mapping from tag names to indices.
        max_len (int): Maximum sentence length.

    Returns:
        torch.Tensor: Tensor of shape (num_sentences, max_len).
    """
    encoded = []
    for label_seq in labels:
        # Convert tag names to indices, default to 'O' if tag is missing
        encoded_labels = [tag2idx.get(tag, tag2idx["O"]) for tag in label_seq]
        # Pad or truncate to max_len
        if len(encoded_labels) < max_len:
            encoded_labels += [tag2idx["PAD"]] * (max_len - len(encoded_labels))
        else:
            encoded_labels = encoded_labels[:max_len]
        encoded.append(encoded_labels)
    return torch.tensor(encoded, dtype=torch.long)

# Retrieve the uploaded file name
import io
for fname in uploaded.keys():
    file_path = fname
    break

# Load and preprocess
sentences = load_text_file(file_path)
print(f"Number of sentences loaded: {len(sentences)}")
tokenized_sentences = tokenize_sentences(sentences)
print(f"First tokenized sentence: {tokenized_sentences[0]}")

# Encode sentences
X_test_custom = encode_sentences(tokenized_sentences, word2idx, MAX_LEN)
print(f"Encoded test data shape: {X_test_custom.shape}")
Y_test_custom = encode_sentences(tokenized_sentences, tag2idx, MAX_LEN)  # `true_labels` should be a list of tag sequences.
print(f"Encoded test data shape: {Y_test_custom.shape}")


Number of sentences loaded: 296
First 3 sentences: ["Amy Mullins says she never looked up the word disabled to see what she'd find.", 'She says she was born into a world that perceived someone like me to have nothing positive going for them.', 'Her childhood doctor at the AI DuPont Institute in Wilmington, Delaware, opened doors for her, she says.']
Number of sentences loaded: 296
First tokenized sentence: ['Amy', 'Mullins', 'says', 'she', 'never', 'looked', 'up', 'the', 'word', 'disabled', 'to', 'see', 'what', 'she', "'d", 'find', '.']
Encoded test data shape: torch.Size([296, 50])
Encoded test data shape: torch.Size([296, 50])


In [28]:
# -----------------------------
# Step 11: Perform NER on Your Text File (Modified for Unlabeled Data)
# -----------------------------

# Load word2idx and tag2idx dictionaries
with open('word2idx.pkl', 'rb') as f:
    word2idx_loaded = pickle.load(f)

with open('tag2idx.pkl', 'rb') as f:
    tag2idx_loaded = pickle.load(f)

idx2tag_loaded = {v: k for k, v in tag2idx_loaded.items()}

# Initialize the model
model = LSTM_CRF(VOCAB_SIZE, TAGSET_SIZE, EMBEDDING_DIM, HIDDEN_DIM, padding_idx=PAD_IDX)
model.to(device)

# Load the saved model state
model_path = 'custom_lstm_crf_model.pth'  # Ensure this path is correct
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
print("Model loaded successfully!")

# Create DataLoader for test data without labels
def create_dataloader_with_labels(encoded_sentences: torch.Tensor, encoded_labels: torch.Tensor, batch_size: int=32) -> DataLoader:
    dataset = TensorDataset(encoded_sentences, encoded_labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

test_loader = create_dataloader_with_labels(X_test_custom, Y_test_custom, batch_size=BATCH_SIZE)

# Perform NER and retrieve predictions
def perform_ner(model: nn.Module, dataloader: DataLoader, idx2tag: dict, device: torch.device) -> List[List[str]]:
    """
    Perform NER and retrieve predicted tags.

    Args:
        model (nn.Module): Trained LSTM_CRF model.
        dataloader (DataLoader): DataLoader with sentences.
        idx2tag (dict): Mapping from tag indices to tag names.
        device (torch.device): Device to perform computation on.

    Returns:
        List[List[str]]: Predicted tags for each sentence.
    """
    all_preds = []

    with torch.no_grad():
        for batch in dataloader:
            batch_X = batch[0].to(device)
            mask = (batch_X != PAD_IDX).to(device)

            # Get predictions
            preds = model.predict(batch_X, mask)

            # Convert indices to tag names
            for pred in preds:
                pred_tags = [idx2tag.get(idx, "O") for idx in pred]
                all_preds.append(pred_tags)

    return all_preds

predicted_tags_custom = perform_ner(model, test_loader, idx2tag_loaded, device)
print("NER prediction completed!")

# Display NER results without true labels
def display_ner_results(sentences: List[List[str]], predicted_tags: List[List[str]]):
    """
    Display tokens with their predicted NER tags.

    Args:
        sentences (List[List[str]]): Original tokenized sentences.
        predicted_tags (List[List[str]]): Predicted NER tags for each token.
    """
    for i, (sentence, pred_tags) in enumerate(zip(sentences, predicted_tags)):
        print(f"Sentence {i+1}:")
        for token, pred_tag in zip(sentence[:len(pred_tags)], pred_tags):
            print(f"{token:15}\tPredicted: {pred_tag:10}")
        print("\n")

# Display results
display_ner_results(tokenized_sentences, predicted_tags_custom)


<ipython-input-28-f11f2b81c072>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Model loaded successfully!
NER prediction completed!
Sentence 1:
Amy            	Predicted: O         
Mullins        	Predicted: O         
says           	Predicted: O         
she            	Predicted: O         
never          	Predicted: O         
looked         	Predicted: O         
up             	Predicted: O         
the            	Predicted: O         
word           	Predicted: O         
disabled       	Predicted: O         
to             	Predicted: O         
see            	Predicted: O         
what           	Predicted: O         
she            	Predicted: O         
'd             	Predicted: O         
find           	Predicted: O         


Sentence 2:
She            	Predicted: O         
says           	Predicted: B-PER     
she            	Predicted: B-PER     
was            	Predicted: O         
born           	Predicted: O         
into           	Predicted: O         
a              	Predicted: O         
world          	Predicted: O         
that     